In [44]:
import requests
import re
import pandas as pd
import pymysql

In [147]:
# 获取网页
def get_content(url, page):
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Connection': 'keep-alive',
        'Cookie': '__mta=142573197.1621327687267.1621327812659.1621327828244.4; uuid_n_v=v1; uuid=C2001140B7B511EB91C2FBC9F18692AD26DB734137784A2C9A100A3142171C78; _csrf=3ec74841158f75a43b8bb165450d3583cdd0e9428df8e0616d42eb3ff9d25834; _lxsdk_cuid=1797ea928dec8-04038090de538a-37607201-13c680-1797ea928dec8; _lxsdk=C2001140B7B511EB91C2FBC9F18692AD26DB734137784A2C9A100A3142171C78; Hm_lvt_703e94591e87be68cc8da0da7cbd0be2=1621327686; __mta=142573197.1621327687267.1621327687267.1621327687267.1; Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2=1621327826; _lxsdk_s=1797ea928e0-cef-9dc-c26%7C%7C10',
        'Host': 'maoyan.com',
        'Referer': 'https://maoyan.com/board/4?offset=10',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
        'sec-ch-ua-mobile': '?0',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
    }
    params = {
        'offset': page * 10
    }
    res = requests.get(url, params=params, headers=headers)
    if res.status_code == 200:
        return res.text
    return


# 正则表达式解析网页
def parse_html(html):
    items = re.findall(r'<dd>(.*?)</dd>', html, re.S)
    rank = [re.findall(r'<i class.*?board-index.*?>(.*?)</i>', item, re.S)[0] for item in items]
    link = ['https://maoyan.com' + re.findall(r'<i class.*?board-index.*?<a href="(.*?)"', item, re.S)[0] for item in items]
    title = [re.findall(r'title="(.*?)"', item, re.S)[0] for item in items]
    image = [re.findall(r'<img data-src="(.*?)"', item, re.S)[0] for item in items]
    actor = [i[0].strip()[3:] if len(i[0].strip()) > 3 else '' for i in [re.findall(r'class="star">(.*?)</p>', item, re.S) for item in items]]
    release_time = [re.findall('class="releasetime">上映时间：(.*?)</p>', item, re.S)[0].strip() for item in items]
    integer = [re.findall('<i class="integer">(.*?)</i>', item, re.S)[0] if re.findall('<i class="integer">(.*?)</i>', item, re.S) else '' for item in items]
    fraction = [re.findall('<i class="fraction">(.*?)</i>', item, re.S)[0] if re.findall('<i class="fraction">(.*?)</i>', item, re.S) else '' for item in items]
    score = [i + j for i, j in list(zip(integer, fraction))]
    
    return pd.DataFrame({'rank': rank, 'link': link, 'title': title, 'image': image, 
                         'actor': actor, 'release_time':release_time, 'score': score})


# 创建database
def create_database(db):
    conn = pymysql.connect(user='root', passwd='123456', port=3306)
    cur = conn.cursor()
    cur.execute(f'CREATE DATABASE IF NOT EXISTS {db} CHARACTER SET utf8mb4')

    
# 创建table
def create_table(db, table):
    conn = pymysql.connect(user='root', passwd='123456', port=3306, db=db)
    cur = conn.cursor()
    sql = f'CREATE TABLE IF NOT EXISTS {table} (`id` INT NOT NULL AUTO_INCREMENT ,' + \
            ','.join(['`' + col + '`' +' ' + 'VARCHAR(255)' for col in data.columns]) + \
            ', PRIMARY KEY (`id`)' + ')' + ' ENGINE = InnoDB;'
    cur.execute(sql)

    
# 批量插入
def insert_data(df, db, table):
    columns = df.columns
    length = len(columns)
    s = ','.join(['%s'] * length)
    col = ','.join(["`" + item + "`" for item in columns])
    sql = f"""insert into {db}.{table} ({col}) values ({s})"""
    conn = pymysql.connect(user='root', passwd='123456', port=3306, db=db)
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, df.values.tolist())
        conn.commit()
        print('数据写入成功')
    except Exception as e:
        print(e)
        conn.rollback()
    

def store_data(df, db, table):
    create_database(db)
    create_table(db, table)
    insert_data(df, db, table)

In [148]:
url = 'https://maoyan.com/board/4'
db = 'MaoYan'
table = 'MaoYanTop100'
data = pd.DataFrame([])
for page in range(10):
    html = get_content(url, page)
    page_data = parse_html(html)
    data = pd.concat([data, page_data])
store_data(data, db, table)

数据写入成功


In [199]:
# 批量更新
def update_data(df, db, table):
    """insert into MaoYan.Top100 (`id`,`name`,`age`) values (%s,%s,%s) 
    on duplicate key update id=values(id),name=values(name),age=values(age)"""
    columns = df.columns
    length = len(columns)
    s = ','.join(['%s'] * length)
    col = ','.join(["`" + item + "`" for item in columns])
    update = ','.join([item + '=values(' + item + ')' for item in columns])
    sql = f"""insert into {db}.{table} ({col}) values ({s}) on duplicate key update {update}"""
    conn = pymysql.connect(user='root', passwd='123456', port=3306, db=db)
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, df.values.tolist())
        conn.commit()
        print('数据更新成功')
    except Exception as e:
        print(e)
        conn.rollback()

In [200]:
test = {
    'id': ['20120001', '20120002'],
    'name': ['Bob', 'Smith'], 
#     'age': [21, 25]
    'age': [26, 25]
}
test_df = pd.DataFrame(test)
test_df

,id,name,age
0,20120001,Bob,26
1,20120002,Smith,25


In [164]:
insert_data(test_df, 'MaoYan', 'Top100')

数据写入成功


In [201]:
update_data(test_df, 'MaoYan', 'Top100')

insert into MaoYan.Top100 (`id`,`name`,`age`) values (%s,%s,%s) on duplicate key update id=values(id),name=values(name),age=values(age)
数据更新成功
